In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
from google.colab import files
files.upload()

Saving product_id_ncds.csv to product_id_ncds.csv


{'product_id_ncds.csv': b'id\r\n271973414\r\n271966788\r\n216469790\r\n271973414\r\n184059211\r\n123345348\r\n271966786\r\n271972435\r\n184036446\r\n197214029\r\n271967379\r\n197214015\r\n'}

In [3]:
import requests
import pandas as pd
import time
import random
from tqdm import tqdm

cookies = {
    'TIKI_GUEST_TOKEN': 'qrNOiGzhYcCBaw30dE9yf25JMXVkFlAR',
    '_trackity':'0f6a9b4e-5c80-bcba-c8d8-011c25fd7a17',
    '_ga':'GA1.1.1255399462.1701873439',
    '_gcl_au':'1.1.406659848.1701873442',
    '_fbp':'fb.1.1701873442152.1210105370',
    '_hjSessionUser_522327':'eyJpZCI6IjJhMTQyN2RjLTg2ZWItNWVjMS1hMTViLWQ5YTExM2Y0MDk5NSIsImNyZWF0ZWQiOjE3MDE4NzM0NDIxOTksImV4aXN0aW5nIjp0cnVlfQ==',
    'TOKENS':'{%22access_token%22:%22qrNOiGzhYcCBaw30dE9yf25JMXVkFlAR%22}',
    'delivery_zone':'Vk4wMzkwMDYwMDE=',
    'tiki_client_id':'1255399462.1701873439',
    '_hjIncludedInSessionSample_522327':'0',
    '_hjSession_522327':'eyJpZCI6ImYyNzdjZmJlLTQ4NzctNDcxNC1iMmE4LTc2Y2IyZGMwYWQzNSIsImMiOjE3MDQ4MjA2NjE5NDgsInMiOjAsInIiOjAsInNiIjowfQ==',
    '_hjAbsoluteSessionInProgress':'0',
    'amp_99d374':'oYuIHjr049nh6dG42AzwSh...1hjnjlje2.1hjnk02m5.6o.7c.e4',
    '_ga_S9GLR1RQFJ':'GS1.1.1704820658.2.1.1704821965.60.0.0'

}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi-VN,vi;q=0.8,en-US;q=0.5,en;q=0.3',
    'Referer': 'https://tiki.vn/apple-iphone-15-pro-max-p271973414.html?itm_campaign=SRC_YPD_TKA_PLA_UNK_ALL_UNK_UNK_UNK_UNK_X.282877_Y.1865197_Z.3899608_CN.AUTO---Apple-iPhone-15-Pro-Max---2023%2F12%2F06-11%3A13%3A23&itm_medium=CPC&itm_source=tiki-ads&spid=271973473%27,',
    'x-guest-token': 'qrNOiGzhYcCBaw30dE9yf25JMXVkFlAR',
    'Connection': 'keep-alive',
    'TE': 'Trailers',
}

params = {
    'limit': '5',
    'include': 'comments,contribute_info,attribute_vote_summary',
    'sort': 'score|desc,id|desc,stars|all',
    'page': '1',
    'spid': '271973464',
    'product_id': '271973414',
    'seller_id': '1'
}

def comment_parser(json):
    d = dict()
    d['id'] = json.get('id')
    d['title'] = json.get('title')
    d['content'] = json.get('content')
    d['thank_count'] = json.get('thank_count')
    d['customer_id']  = json.get('customer_id')
    d['rating'] = json.get('rating')
    d['created_at'] = json.get('created_at')
    d['customer_name'] = json.get('created_by').get('name')
    d['purchased_at'] = json.get('created_by').get('purchased_at')
    return d




In [4]:
BASE_URL = 'https://tiki.vn/api/v2/reviews'

def crawl_comments(product_id):
    result = []
    for page in range(10):  # Adjust the number of pages accordingly
        params['page'] = page
        response = make_request(product_id)
        if response.status_code == 200:
            for comment in response.json().get('data'):
                result.append(comment_parser(comment))
        time.sleep(1)  # Add a delay to avoid rate limits
    return result

def make_request(product_id):
    params['product_id'] = product_id
    try:
        response = requests.get(BASE_URL, headers=headers, params=params, cookies=cookies)
        response.raise_for_status()
        return response
    except requests.exceptions.RequestException as e:
        print(f"Error making request for product {product_id}: {e}")
        return None

# Main script
df_id = pd.read_csv('/content/product_id_ncds.csv')
p_ids = df_id.id.tolist()
result = []

for pid in tqdm(p_ids, total=len(p_ids)):
    print(f'Crawling comments for product {pid}')
    comments = crawl_comments(pid)
    result.extend(comments)

df_comment = pd.DataFrame(result)
df_comment.to_csv('comments_data_ncds.csv', index=False, encoding='utf-8-sig')
df_comment

  0%|          | 0/12 [00:00<?, ?it/s]

Crawling comments for product 271973414


  8%|▊         | 1/12 [00:14<02:41, 14.68s/it]

Crawling comments for product 271966788


 17%|█▋        | 2/12 [00:28<02:23, 14.34s/it]

...

Crawling comments for product 271967379


 92%|█████████▏| 11/12 [02:45<00:15, 15.20s/it]

Crawling comments for product 197214015


100%|██████████| 12/12 [03:00<00:00, 15.03s/it]


,id,title,content,thank_count,customer_id,rating,created_at,customer_name,purchased_at
0,19559920,Cực kì hài lòng,"22/9 đúng 00h lên topzone, TGDĐ, FPT Shop, đặt...",21,945720,5,1696038016,muasach,1695922721
1,19562673,Cực kì hài lòng,Đam mê táo k thể bỏ qua Iphone 15promax vì: \r...,10,1023265,5,1696145764,Vũ thị hường,1695923820
2,19557915,Cực kì hài lòng,[Cuộc gọi lúc giữa đêm]…\r\n\r\nKhông phải bon...,15,1670022,5,1695927222,BẢO NGÔ,1695920389
3,19557927,Cực kì hài lòng,Tiki giao lúc 1h sáng. Áp mã 500K và giảm 2tr ...,8,49454,5,1695934375,NGÔ QUỐC KHÁNH,1695920576
4,19614635,Cực kì hài lòng,Rất tin tưởng Tiki khi đặt mua điện thoại tại ...,2,756369,5,1698395066,Hung Trieu,1698341954
...,...,...,...,...,...,...,...,...,...
467,19697709,Cực kì hài lòng,SP ok,0,989179,5,1703648409,Nguyễn Minh Hiếu,1703306866
468,18105392,Cực kì hài lòng,,1,12989530,5,1667327157,Tran Manh,1666715147
469,18808961,Cực kì hài lòng,"Sản phẩm đẹp, giao nhanh, cẩn thận, uy tín",0,9043061,5,1676484748,Hà Trang,1672894924
470,19696676,Cực kì hài lòng,,0,10413260,5,1703585126,Phạm Bảo Nhật,1703561808
